# Getting data

In this notebook, we scrape and combine data related to the last 6 seasons of Bundesliga from [https://fbref.com](https://fbref.com).

We'll use the resulting csv file for building our model in a separate notebook.

In [1]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

We start with the main page of Fußball-Bundesliga, and pick the urls for the last 6 seasons.

In [2]:
url = "https://fbref.com/en/comps/20/history/Bundesliga-Seasons"

data = requests.get(url)
soup = BeautifulSoup(data.text)
seasons = soup.select('table.stats_table', id='seasons')[0]

links = seasons.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if '/comps/' in l]

In [3]:
links = list(set(links))
links.sort(reverse=True)

# Pick last 6 seasons
season_urls = [f"https://fbref.com{l}" for l in links[1:7]]
season_urls

['https://fbref.com/en/comps/20/2022-2023/2022-2023-Bundesliga-Stats',
 'https://fbref.com/en/comps/20/2021-2022/2021-2022-Bundesliga-Stats',
 'https://fbref.com/en/comps/20/2020-2021/2020-2021-Bundesliga-Stats',
 'https://fbref.com/en/comps/20/2019-2020/2019-2020-Bundesliga-Stats',
 'https://fbref.com/en/comps/20/2018-2019/2018-2019-Bundesliga-Stats',
 'https://fbref.com/en/comps/20/2017-2018/2017-2018-Bundesliga-Stats']

Next we write a function that looks at the stats table for each season, and picks the urls for each team.

In [4]:
def get_team_urls(url):
    '''Returns the team urls given season url'''    
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    season_table = soup.select('table.stats_table')[0]

    links = season_table.find_all('a')
    links = [l.get('href') for l in links]
    links = [l for l in links if '/squads/' in l]

    return [f"https://fbref.com{l}" for l in links]

Each team page contains tables with different match log types. We will start with the Scores & Fixtures table which is directly on the team page, and we will pick some additional columns, listed below, from several tables which are accessed via links on the team page. Explanations for each column can be found on the relevant webpage of each team.

- Shooting - columns: `'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt'`
- Goalkeeping - columns: `'SoTA', 'GA', 'Saves', 'PKA', 'PKsv'`
- Passing - columns: `'Cmp', 'Att', 'TotDist', 'PrgDist'`
- Pass Types - columns`'Live', 'Dead', 'TB', 'Sw', 'Crs', 'TI', 'CK'`

In [5]:
def get_scores(url):
    data = requests.get(url)
    return pd.read_html(data.text, match='Scores & Fixtures')[0]

In [6]:
def get_tables(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get('href') for l in links]

    links_shooting = [l for l in links if l and '/all_comps/shooting/' in l]

    time.sleep(2)
    data = requests.get(f"https://fbref.com{links_shooting[0]}")
    shooting = pd.read_html(data.text, match='Shooting')[0]
    shooting.columns = shooting.columns.droplevel()

    links_goalkeeping = [l for l in links if l and '/all_comps/keeper/' in l]

    time.sleep(2)
    data = requests.get(f"https://fbref.com{links_goalkeeping[0]}")
    goalkeeping = pd.read_html(data.text, match='Goalkeeping')[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()

    links_passing = [l for l in links if l and '/all_comps/passing/' in l]

    time.sleep(2)
    data = requests.get(f"https://fbref.com{links_passing[0]}")
    passing = pd.read_html(data.text, match='Passing')[0]
    passing.columns = passing.columns.droplevel()

    links_passing_types = [l for l in links if l and '/all_comps/passing_types/' in l]

    time.sleep(2)
    data = requests.get(f"https://fbref.com{links_passing_types[0]}")
    passing_types = pd.read_html(data.text, match='Pass Types')[0]
    passing_types.columns = passing_types.columns.droplevel()

    return shooting, goalkeeping, passing, passing_types

In [7]:
dfs = []

for season_url in season_urls:
    team_urls = get_team_urls(season_url)

    for team_url in team_urls:
        scores = get_scores(team_url)
        shooting, goalkeeping, passing, passing_types = get_tables(team_url)
        try:
            team_df = scores.merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on='Date')
            team_df = team_df.merge(goalkeeping[['Date', 'SoTA', 'Saves', 'PKA', 'PKsv']], on='Date')
            team_df = team_df.merge(passing[['Date', 'Cmp', 'Att', 'TotDist', 'PrgDist']], on='Date')
            # There are several Cmp and Att columns in passing,
            # only keep the first ones
            team_df = team_df.loc[:, ~team_df.columns.duplicated()]
            team_df = team_df.merge(passing_types[['Date', 'Live', 'Dead', 'TB', 'Sw', 'Crs', 'TI', 'CK']], on='Date')
        except ValueError:
            continue

        team_df = team_df[team_df['Comp'] == 'Bundesliga']
        team_df['season'] = season_url.split('/')[-2]
        team_df['team'] = team_url.split('/')[-1].replace('-Stats', '').replace('-', ' ')

        dfs.append(team_df)
        time.sleep(5)

In [8]:
df = pd.concat(dfs)
df.columns = [c.lower() for c in df.columns]

In [9]:
df.shape

(3672, 42)

In [10]:
pd.set_option('display.max_columns', None)
df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,attendance,captain,formation,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,sota,saves,pka,pksv,cmp,att,totdist,prgdist,live,dead,tb,sw,crs,ti,ck,season,team
1,2022-08-05,20:30,Bundesliga,Matchweek 1,Fri,Away,W,6,1,Eint Frankfurt,4.0,0.9,63.0,51500.0,Manuel Neuer,4-4-2,Deniz Aytekin,Match Report,NaN,23.0,10.0,16.6,1.0,0.0,0.0,2.0,1.0,0.0,0.0,572.0,661.0,10072.0,3374.0,623.0,35.0,7.0,4.0,14.0,11.0,6.0,2022-2023,Bayern Munich
2,2022-08-14,17:30,Bundesliga,Matchweek 2,Sun,Home,W,2,0,Wolfsburg,2.0,0.5,68.0,75000.0,Manuel Neuer,4-2-3-1,Harm Osmers,Match Report,NaN,24.0,8.0,18.4,0.0,0.0,0.0,2.0,2.0,0.0,0.0,549.0,648.0,9680.0,3508.0,592.0,50.0,1.0,5.0,20.0,23.0,8.0,2022-2023,Bayern Munich
3,2022-08-21,17:30,Bundesliga,Matchweek 3,Sun,Away,W,7,0,Bochum,2.9,0.7,64.0,26000.0,Manuel Neuer,4-4-2,Daniel Siebert,Match Report,NaN,21.0,11.0,18.3,1.0,1.0,1.0,7.0,7.0,0.0,0.0,462.0,545.0,7943.0,3056.0,498.0,42.0,3.0,2.0,10.0,16.0,5.0,2022-2023,Bayern Munich
4,2022-08-27,18:30,Bundesliga,Matchweek 4,Sat,Home,D,1,1,M'Gladbach,2.8,0.5,69.0,75000.0,Manuel Neuer,4-2-2-2,Daniel Schlager,Match Report,NaN,33.0,20.0,17.4,1.0,0.0,0.0,3.0,2.0,0.0,0.0,576.0,675.0,9943.0,3614.0,621.0,47.0,1.0,7.0,33.0,20.0,14.0,2022-2023,Bayern Munich
6,2022-09-03,15:30,Bundesliga,Matchweek 5,Sat,Away,D,1,1,Union Berlin,1.5,0.3,74.0,22012.0,Manuel Neuer,4-2-3-1,Frank Willenborg,Match Report,NaN,21.0,6.0,18.1,0.0,0.0,0.0,2.0,1.0,0.0,0.0,657.0,762.0,10395.0,3464.0,710.0,50.0,6.0,2.0,17.0,17.0,7.0,2022-2023,Bayern Munich


In [11]:
df.drop(['comp', 'notes', 'match report'], axis=1, inplace=True)

In [12]:
df.to_csv('match_data.csv')